In [2]:
import pandas as pd
import json

In [3]:
df = pd.read_csv('movies_dataset.csv')

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3176\259290390.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_dataset.csv')


<h3> Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,vote_count,poster_path y homepage.

In [4]:
movies = df.drop(columns=['video', 'homepage', 'poster_path', 'imdb_id', 'adult', 'original_title', 'vote_count'])

<h3> Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

Relleno con 0 los valores nulos en las columnas 'bugdet' y 'revenue', para poder calcular el 'return'

In [5]:
movies.fillna({'budget':0, 'revenue': 0}, inplace=True)

In [6]:
#No se puede convertir la columna 'budget' a tipo numérico porque hay registros que tienen todos sus valores incorrectos, así que los eliminaremos
movies[movies['budget']== '/ff9qCepilowshEtG2GYWwzt2bs4.jpg']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average
19730,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...",1997-08-20,104.0,Released,NaN,False,6.0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
movies[movies['budget']== '/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average
29503,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...",2012-09-29,68.0,Released,NaN,False,7.0,12,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
movies[movies['budget']== '/zaSf5OG7V8X8gqFvly88zDdRm46.jpg']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average
35587,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...",2014-01-01,82.0,Released,Beware Of Frost Bites,False,4.3,22,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
movies = movies.drop(19730)

In [10]:
movies = movies.drop(29503)

In [11]:
movies= movies.drop(35587)

In [12]:
#Sin esos registros erróneos, se puede cambiar el tipo a numérico
movies['budget'] = movies['budget'].astype('float64')

Calculo el 'return' a partir de (revenue / budget). Si alguno de estos valores es 0, 'return' será también 0

In [13]:
#Defino una función que primero comprobará si alguno de los valores es 0, y si no, retornará el cálculo
def calculo_return(x, y):
    if (x == 0).any() or (y == 0).any():
        return 0
    else:
        return x / y

In [14]:
#Aplico la función anterior al dataframe, retornando los valores calculados en la columna 'return'
movies['return'] = movies.apply(lambda x: calculo_return(pd.Series(x['revenue']), pd.Series(x['budget'])), axis=1)

<h3> De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [15]:
#Reemplazo los guiones por barras para poder hacer la conversión
movies['release_date'] = movies['release_date'].str.replace("-", '/')

In [16]:
#Convierto a formato fecha y le doy el formato que se solicita
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')

In [17]:
#Extraigo el año de la fecha ya convertida
movies['release_year'] = movies['release_date'].dt.year

In [18]:
#Cambio el tipo de dato del año a int, previamente reemplazando los valores faltantes por 0
movies.fillna({'release_year':0}, inplace=True)
movies['release_year'] = movies['release_year'].astype('int')

In [19]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4491 non-null   object        
 1   budget                 45463 non-null  float64       
 2   genres                 45463 non-null  object        
 3   id                     45463 non-null  object        
 4   original_language      45452 non-null  object        
 5   overview               44509 non-null  object        
 6   popularity             45460 non-null  object        
 7   production_companies   45460 non-null  object        
 8   production_countries   45460 non-null  object        
 9   release_date           45376 non-null  datetime64[ns]
 10  revenue                45463 non-null  float64       
 11  runtime                45203 non-null  float64       
 12  spoken_languages       45460 non-null  object        
 13  s

In [20]:
#Antes de empezar a desanidar las columnas, hago transformaciones de tipos de datos a las que lo necesitan
movies['popularity'] = movies['popularity'].astype('float64')

<h3> Los valores nulos del campo release date deben eliminarse.

In [21]:
movies.drop(movies[movies['release_date'].isnull()].index, inplace=True)

<h3> Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [22]:
#importo la librería necesaria
import ast

In [23]:
#Defino una función que permitirá extraer los valores que se encuentran anidados y colocará dentro de una lista solo los valores de la clave 'name', ignorando los ID's de los géneros, países, etc.
def aplanar(columna):
    L = []
    for i in ast.literal_eval(columna):
        L.append(i['name'])
    return L

In [24]:
movies['genres'] = movies['genres'].apply(aplanar)
movies['production_companies'] = movies['production_companies'].apply(aplanar)
movies['production_countries'] = movies['production_countries'].apply(aplanar)
movies['spoken_languages'] = movies['spoken_languages'].apply(aplanar)


In [25]:
#Reemplazo las comillas simples para poder leerlo con json.loads
movies['belongs_to_collection'] = movies['belongs_to_collection'].str.replace("'", '"')

In [26]:
#Defino la función para retornar un diccionario vacío en caso de error y crear un dataframe a partir de los valores del diccionario
def excepcion_json(columna):
    try:
        return json.loads(columna)
    except:
        return {}

movies['belongs_to_collection'] = movies['belongs_to_collection'].fillna('{}')
movies_belongs = pd.json_normalize(movies['belongs_to_collection'].apply(excepcion_json))


In [27]:
#Convierto a DataFrame, voy a quedarme únicamente con el nombre de la colección, no necesito los paths de las imágenes ni los IDs
movies_belongs = pd.DataFrame(movies_belongs)

In [28]:
movies['belongs_to_collection'] = movies_belongs['name']

In [29]:
#Aplano las listas que quedaron en estas columnas
movies['genres'] = movies['genres'].apply(lambda x: ', '.join(map(str, x)))
movies['production_companies'] = movies['production_companies'].apply(lambda x: ', '.join(map(str, x)))
movies['production_countries'] = movies['production_countries'].apply(lambda x: ', '.join(map(str, x)))
movies['spoken_languages'] = movies['spoken_languages'].apply(lambda x: ', '.join(map(str, x)))

In [30]:
movies.to_csv("movies_limpio.csv", index=False)